In [136]:
from pathlib import Path
import numpy as np
import pandas as pd

In [137]:
input = Path("example.txt").read_text()
print(input)

125 17



In [138]:
stones = np.array(input.split(), dtype=np.int64)
print(stones)

[125  17]


In [139]:
def split(stone: int) -> tuple[int] | tuple[int, int]:
    if stone == 0:
        return (1,)
    log_10 = int(np.floor(np.log10(stone)))
    if log_10 % 2 == 1:
        median = 10 ** ((log_10 + 1) // 2)
        return (stone // median, stone % median)
    return (stone * 2024,)

In [140]:
# Solution 1
num_blinks = 25
new_stones = stones
print("Inital arrangement:")
print(new_stones)
for i in range(num_blinks):
    new_stones = [s for stone in new_stones for s in split(stone) if s is not None]
    print(f"After {i + 1} blink{'s' if i > 0 else ''}:")
    print(new_stones)
print(len(new_stones), "stones!")

Inital arrangement:
[125  17]
After 1 blink:
[253000, 1, 7]
After 2 blinks:
[253, 0, 2024, 14168]
After 3 blinks:
[512072, 1, 20, 24, 28676032]
After 4 blinks:
[512, 72, 2024, 2, 0, 2, 4, 2867, 6032]
After 5 blinks:
[1036288, 7, 2, 20, 24, 4048, 1, 4048, 8096, 28, 67, 60, 32]
After 6 blinks:
[2097446912, 14168, 4048, 2, 0, 2, 4, 40, 48, 2024, 40, 48, 80, 96, 2, 8, 6, 7, 6, 0, 3, 2]
After 7 blinks:
[20974, 46912, 28676032, 40, 48, 4048, 1, 4048, 8096, 4, 0, 4, 8, 20, 24, 4, 0, 4, 8, 8, 0, 9, 6, 4048, 16192, 12144, 14168, 12144, 1, 6072, 4048]
After 8 blinks:
[42451376, 94949888, 2867, 6032, 4, 0, 4, 8, 40, 48, 2024, 40, 48, 80, 96, 8096, 1, 8096, 16192, 2, 0, 2, 4, 8096, 1, 8096, 16192, 16192, 1, 18216, 12144, 40, 48, 32772608, 24579456, 28676032, 24579456, 2024, 60, 72, 40, 48]
After 9 blinks:
[4245, 1376, 9494, 9888, 28, 67, 60, 32, 8096, 1, 8096, 16192, 4, 0, 4, 8, 20, 24, 4, 0, 4, 8, 8, 0, 9, 6, 80, 96, 2024, 80, 96, 32772608, 4048, 1, 4048, 8096, 80, 96, 2024, 80, 96, 32772608, 327

In [141]:
# Solution 2
num_blinks = 75
stone_df = pd.DataFrame(stones, columns=["stone"]).assign(count=1)
for i in range(num_blinks):
    stone_df = (
        stone_df.assign(split=lambda df: df["stone"].apply(split))
        .explode("split")
        .assign(stone=lambda df: df["split"])
        .drop(columns="split")
        .groupby("stone")
        .sum()
        .reset_index()
    )
stone_df

,stone,count
0,0,3389584081458
1,1,2436251302573
2,2,3500884482938
3,3,860836234395
4,4,3709479394908
5,5,727285152014
6,6,2940934071154
7,7,1717175622986
8,8,3621875746005
9,9,1622336990432


In [142]:
stone_df["count"].sum()

65601038650482